In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

from loader import train_loader, val_loader, test_loader, scaler
from models.pft import ProbabilisticForecastTransformer

In [ ]:
def distributional_vae_loss(out, y_true, mu_z, logvar_z, kl_weight=0.001):
    # out: (batch, output_dim*2) -> tách ra μ_y và logvar_y
    batch_size, out_dim2 = out.shape
    output_dim = out_dim2 // 2
    mu_y = out[:, :output_dim]
    logvar_y = out[:, output_dim:]
    
    sigma_y = torch.exp(0.5 * logvar_y)
    nll = 0.5 * (np.log(2 * np.pi) + logvar_y + ((y_true - mu_y)**2 / (sigma_y**2)))
    nll = torch.mean(torch.sum(nll, dim=1))
    
    kl = -0.5 * torch.mean(1 + logvar_z - mu_z.pow(2) - logvar_z.exp())
    
    return nll + kl_weight * kl

In [ ]:
model = ProbabilisticForecastTransformer(
	window_size=30, 
	num_series=2, 
	static_dim=18,
    latent_dim=32, 
	d_model=64, 
	nhead=4, 
	num_layers=2,
    hidden_dim=128, 
	dropout=0.1, 
	output_dim=2
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [35]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
kl_start = 0.0
kl_max = 0.001

In [36]:
epochs = 100
train_losses = []
val_losses = []
best_val_loss = float('inf')

# KL annealing: tăng kl_weight dần từ 0 đến 0.001
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    kl_weight = min(0.001, 0.001 * (epoch / 50))
    for x_seq, x_cal, y in train_loader:
        x_seq = x_seq.to(device)
        x_cal = x_cal.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        out, mu_z, logvar_z = model(x_seq, x_cal)
        loss = distributional_vae_loss(out, y, mu_z, logvar_z, kl_weight=kl_weight)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item() * x_seq.size(0)
    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)
    
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for x_seq, x_cal, y in val_loader:
            x_seq = x_seq.to(device)
            x_cal = x_cal.to(device)
            y = y.to(device)
            out, mu_z, logvar_z = model(x_seq, x_cal)
            loss = distributional_vae_loss(out, y, mu_z, logvar_z, kl_weight=kl_weight)
            running_val_loss += loss.item() * x_seq.size(0)
    epoch_val_loss = running_val_loss / len(val_loader.dataset)
    val_losses.append(epoch_val_loss)
    
    scheduler.step(epoch_val_loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, KL Weight: {kl_weight:.6f}")
    
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        checkpoint = {
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': epoch_val_loss,
        }
        torch.save(checkpoint, 'best_probabilistic_vae_checkpoint.pth')
        print(f"Best model updated at epoch {epoch+1} with validation loss {epoch_val_loss:.4f}")

Best model updated at epoch 1 with validation loss 1.6217
Best model updated at epoch 2 with validation loss 1.1114
Best model updated at epoch 3 with validation loss 0.8380
Best model updated at epoch 5 with validation loss 0.6951
Best model updated at epoch 6 with validation loss 0.6185
Best model updated at epoch 7 with validation loss 0.6158
Best model updated at epoch 8 with validation loss 0.3470
Epoch [10/100], Train Loss: 0.2992, Val Loss: 0.2478, KL Weight: 0.000180
Best model updated at epoch 10 with validation loss 0.2478
Best model updated at epoch 11 with validation loss 0.2406
Best model updated at epoch 12 with validation loss 0.1906
Best model updated at epoch 13 with validation loss 0.1583
Best model updated at epoch 15 with validation loss 0.0474
Best model updated at epoch 16 with validation loss 0.0266
Best model updated at epoch 18 with validation loss -0.0079
Epoch [20/100], Train Loss: -0.2334, Val Loss: -0.0446, KL Weight: 0.000380
Best model updated at epoch 20

In [37]:
# Evaluation on test set
model.eval()
test_preds = []
test_actuals = []
with torch.no_grad():
    for x_seq, x_cal, y in test_loader:
        x_seq = x_seq.to(device)
        x_cal = x_cal.to(device)
        y = y.to(device)
        out, mu_z, logvar_z = model(x_seq, x_cal)
        # Lấy phần μ của dự báo (mu_y) từ output
        out_dim2 = out.shape[1]
        out_dim = out_dim2 // 2
        mu_y = out[:, :out_dim]  # (batch, output_dim)
        test_preds.append(mu_y.cpu().numpy())
        test_actuals.append(y.cpu().numpy())

test_preds = np.concatenate(test_preds, axis=0)
test_actuals = np.concatenate(test_actuals, axis=0)

# Giả sử bạn có scaler để inverse transform target
test_preds_inv = scaler.inverse_transform(test_preds)
test_actuals_inv = scaler.inverse_transform(test_actuals)

r2 = r2_score(test_actuals_inv, test_preds_inv)
mape = mean_absolute_percentage_error(test_actuals_inv, test_preds_inv)
rmse = np.sqrt(mean_squared_error(test_actuals_inv, test_preds_inv))
print(f"Test R-squared: {r2:.4f}")
print(f"Test MAPE: {mape:.4f}")
print(f"Test RMSE: {rmse:.4f}")

Test R-squared: 0.9682
Test MAPE: 0.2565
Test RMSE: 110592.9688


In [38]:
# Đánh giá trên Units
r2_units = r2_score(test_actuals_inv[:, 0], test_preds_inv[:, 0])
mape_units = mean_absolute_percentage_error(test_actuals_inv[:, 0], test_preds_inv[:, 0])
rmse_units = np.sqrt(mean_squared_error(test_actuals_inv[:, 0], test_preds_inv[:, 0]))

print(f"Test Units - R-squared: {r2_units:.4f}")
print(f"Test Units - MAPE: {mape_units:.4f}")
print(f"Test Units - RMSE: {rmse_units:.4f}")

Test Units - R-squared: 0.9599
Test Units - MAPE: 0.3017
Test Units - RMSE: 25.8991


In [39]:
# Đánh giá trên Revenue
r2_revenue = r2_score(test_actuals_inv[:, 1], test_preds_inv[:, 1])
mape_revenue = mean_absolute_percentage_error(test_actuals_inv[:, 1], test_preds_inv[:, 1])
rmse_revenue = np.sqrt(mean_squared_error(test_actuals_inv[:, 1], test_preds_inv[:, 1]))

print(f"Test Revenue - R-squared: {r2_revenue:.4f}")
print(f"Test Revenue - MAPE: {mape_revenue:.4f}")
print(f"Test Revenue - RMSE: {rmse_revenue:.4f}")

Test Revenue - R-squared: 0.9764
Test Revenue - MAPE: 0.2112
Test Revenue - RMSE: 156402.0625
